#0rders data table creation

In [0]:
create or refresh streaming table bronze_orders
comment 'raw orders data ingested from the source system to operational data'
tblproperties ('quality' = 'bronze')
AS 
select *,
        current_timestamp() as ingestion_ts,
        _metadata.file_path as input_file_path
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/orders/', 'json',
  map('cloudfiles.inferColumnTypes', 'true')
);

Name,Type
customer_id,bigint
items,"array<struct<category:string,item_id:bigint,name:string,price:bigint,quantity:bigint>>"
order_id,bigint
order_status,string
order_timestamp,string
payment_method,string
_rescued_data,string
ingestion_ts,timestamp
input_file_path,string


In [0]:
create or refresh streaming table silver_orders_clean (
     
    constraint valid_customer_id EXPECT (customer_id is not null) on violation fail update,
    constraint valid_order_id EXPECT (order_id is not null) on violation fail update,
    constraint valid_order_status EXPECT (order_status in('Pending', 'Shipped', 'Cancelled' , 'Completed')),
    constraint valid_payment_method EXPECT (payment_method in('Credit Card', 'PayPal', 'Bank Transfer'))

)

comment 'Cleaned orders data'
tblproperties ('quality' = 'silver')
As 
select order_id,
       customer_id,
       Cast (order_timestamp as timestamp) as order_timestamp,
       payment_method,
       items,
       order_status
    from stream(live.bronze_orders) 

Name,Type
order_id,bigint
customer_id,bigint
order_timestamp,timestamp
payment_method,string
items,"array<struct<category:string,item_id:bigint,name:string,price:bigint,quantity:bigint>>"
order_status,string


#exploding orders__data 

In [0]:
create or refresh streaming table silver_orders
AS 
with exploded_data as (
select order_id,
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
      explode(items) as item
   from stream(live.silver_orders_clean)
)   
    
    Select  order_id,
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
      item.item_id,
       item.name as item_name,
       item.price as item_price,
       item.category as item_category,
       item.quantity as item_quantity
     
    from exploded_data;
       
  
       

Name,Type
order_id,bigint
customer_id,bigint
order_timestamp,timestamp
payment_method,string
order_status,string
item_id,bigint
item_name,string
item_price,bigint
item_category,string
item_quantity,bigint
